In [129]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path='./Data/'
df=pd.read_csv(data_path+'titanic_train.csv')

train_Y=df['Survived']
df=df.drop(['PassengerId','Survived'],axis=1)
df.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [130]:
#D29-1
grouped=df['Fare'].groupby(df['Cabin'])
mean_df=grouped.mean()
#mode_df=grouped.apply(lambda x:x.mode())
median_df=grouped.median()
max_df=grouped.max()
min_df=grouped.min()

temp=pd.merge(mean_df,median_df,how='left',on=['Cabin'])
temp=pd.merge(temp,max_df,how='left',on=['Cabin'])
temp=pd.merge(temp,min_df,how='left',on=['Cabin'])
temp.columns=['Fare_Mean','Fare_Median','Fare_Max','Fare_Min']
print(temp)

        Fare_Mean  Fare_Median  Fare_Max  Fare_Min
Cabin                                             
A10     40.125000     40.12500   40.1250   40.1250
A14     52.000000     52.00000   52.0000   52.0000
A16     39.600000     39.60000   39.6000   39.6000
A19     26.000000     26.00000   26.0000   26.0000
A20     56.929200     56.92920   56.9292   56.9292
A23     30.000000     30.00000   30.0000   30.0000
A24     50.495800     50.49580   50.4958   50.4958
A26     35.500000     35.50000   35.5000   35.5000
A31     31.000000     31.00000   31.0000   31.0000
A32     50.000000     50.00000   50.0000   50.0000
A34     81.858300     81.85830   81.8583   81.8583
A36      0.000000      0.00000    0.0000    0.0000
A5      34.654200     34.65420   34.6542   34.6542
A6      35.500000     35.50000   35.5000   35.5000
A7      30.695800     30.69580   30.6958   30.6958
B101   512.329200    512.32920  512.3292  512.3292
B102     0.000000      0.00000    0.0000    0.0000
B18     57.979200     57.97920 

In [131]:
df=pd.merge(df,temp,how='left',on=['Cabin'])
df=df.drop(['Cabin'],axis=1)

df.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_Mean,Fare_Median,Fare_Max,Fare_Min
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,NaN,NaN,NaN
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.2833,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,NaN,NaN,NaN
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.1000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,NaN,NaN,NaN


In [132]:
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Median', 'Fare_Max', 'Fare_Min']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Median,Fare_Max,Fare_Min
0,3,22.0,1,0,7.2500,-1.0000,-1.0000,-1.0000,-1.0000
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,-1.0000,-1.0000,-1.0000,-1.0000
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,-1.0000,-1.0000,-1.0000,-1.0000


In [133]:
#D29-2, Oringinal features with Logistic Reg

df_minus=df.drop(['Fare_Mean','Fare_Median','Fare_Max','Fare_Min'],axis=1)

train_X=MMEncoder.fit_transform(df_minus)
estimator=LogisticRegression()
print(f' Original features with Logistic Reg:{cross_val_score(estimator,train_X,train_Y,cv=5).mean()}\n')


 Original features with Logistic Reg:0.7038635542329971



In [134]:
#D29-2, new features with Logistic Reg
train_X=MMEncoder.fit_transform(df)
print(f' New features with Logistic Reg:{cross_val_score(estimator,train_X,train_Y,cv=5).mean()}\n')


 New features with Logistic Reg:0.7016164341491042

